In [1]:
import os
import sys
import comtypes.client


AttachToInstance = True

# Create API helper object
helper = comtypes.client.CreateObject('SAP2000v1.Helper')
helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)

mySapObject = helper.GetObject("CSI.SAP2000.API.SapObject")

# Create SapModel object
SapModel = mySapObject.SapModel

In [2]:
#initialize model

SapModel.InitializeNewModel()

#create new blank model

ret = SapModel.File.NewBlank()

In [3]:
#Materiales en Mpa
ResistenciaConcreto = 28
Poisson = 0.2
Retraccion = 9.900E-06
Elasticity = 4700*(ResistenciaConcreto)**0.5

NombreMaterial = "C28"

#Cambiar Unidades switch to k-ft units

N_mm_C = 9

ret = SapModel.SetPresentUnits(N_mm_C)

#Define material properties
MATERIAL_CONCRETE = 2

ret = SapModel.PropMaterial.SetMaterial('C28', MATERIAL_CONCRETE)

#assign isotropic mechanical properties to material

ret = SapModel.PropMaterial.SetMPIsotropic('C28', Elasticity, Poisson, Retraccion)

In [4]:
#Cambiar Unidades switch to kN-m-C units

kN_m_C = 6

ret = SapModel.SetPresentUnits(kN_m_C)


In [5]:
#Definimos espesores para los elementos en metros
tLosaSuperior = 0.25
tLosaInferior = 0.25
tMuros = 0.25
NombreMaterial = "C28"
#define rectangular frame section property

ret = SapModel.PropFrame.SetRectangle('LosaSuperior', NombreMaterial, tLosaSuperior, 1)
ret = SapModel.PropFrame.SetRectangle('LosaInferior', NombreMaterial, tLosaInferior, 1)
ret = SapModel.PropFrame.SetRectangle('Muro', NombreMaterial, tMuros, 1)


#define frame section property modifiers

#ModValue = [1, 0, 0, 1, 1, 1, 1, 1]

#ret = SapModel.PropFrame.SetModifiers('LosaSuperior', ModValue)

In [ ]:
#Definir dimensiones

NumeroAlcantarillas = 2
SS = 3.20
H2 = 3.5

#add frame object by coordinates

for i in enumerate NumeroAlcantarillas+1
 FrameName1[] = ' '
 [FrameName1, ret] = SapModel.FrameObj.AddByCoord(0, 0, 0, SS*i, 0, 10, FrameName1, 'Seccion', '1', 'Global')


FrameName2 = ' '

FrameName3 = ' '

[FrameName1, ret] = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, 10, FrameName1, 'LosaSuperior', '1', 'Global')

[FrameName2, ret] = SapModel.FrameObj.AddByCoord(0, 0, 10, 8, 0, 16, FrameName2, 'LosaSuperior', '2', 'Global')

[FrameName3, ret] = SapModel.FrameObj.AddByCoord(-4, 0, 10, 0, 0, 10, FrameName3, 'LosaSuperior', '3', 'Global')

In [7]:
num_culverts = 2
span_width = 1.2
height = 2

# Generate frames for portal culvert
for i in range(num_culverts + 1):
    # Bottom horizontal members
    SapModel.FrameObj.AddByCoord(
    x1=0, y1=i*span_width, z1=0, 
    x2=span_width, y2=i*span_width, z2=0, 
    PropName='LosaInferior',"losa1","Global"
    )
            
    # Vertical members
    SapModel.FrameObj.AddByCoord(
    x1=0, y1=i*span_width, z1=0, 
    x2=0, y2=i*span_width, z2=height, 
    PropName='Muro'
    )
            
    SapModel.FrameObj.AddByCoord(
    x1=span_width, y1=i*span_width, z1=0, 
    x2=span_width, y2=i*span_width, z2=height, 
    PropName='Muro'
    )
            
    # Top horizontal members
    SapModel.FrameObj.AddByCoord(
    x1=0, y1=i*span_width, z1=height, 
    x2=span_width, y2=i*span_width, z2=height, 
    PropName='LosaSuperior'
    )
        

SyntaxError: positional argument follows keyword argument (880976787.py, line 12)

In [10]:
NumeroAlcantarillas = 3
SS=3
H2 = 2
# Crear pórticos de alcantarillas
for i in range(NumeroAlcantarillas):
    X_start = SS * i
    X_end = X_start + SS
    
    # Definir coordenadas de los marcos
    _, FrameName1 = SapModel.FrameObj.AddByCoord(X_start, 0, H2, X_end, 0, H2, '', 'LosaSuperior', 'Global')  # Losa superior
    _, FrameName2 = SapModel.FrameObj.AddByCoord(X_start, 0, 0, X_end, 0, 0, '', 'LosaInferior', 'Global')  # Losa inferior
    _, FrameName3 = SapModel.FrameObj.AddByCoord(X_start, 0, 0, X_start, 0, H2, '', 'Muro', 'Global')  # Columna izquierda
    _, FrameName4 = SapModel.FrameObj.AddByCoord(X_end, 0, 0, X_end, 0, H2, '', 'Muro', 'Global')    # Columna derecha




In [11]:
# Nombre de la propiedad que deseas buscar
property_name = "LosaInferior"

# Crear un grupo con todos los elementos que usan la propiedad "LosaSuperior"
group_name = "GrupoLosaInferior"
ret = SapModel.GroupDef.SetGroup(group_name)
ret = SapModel.SelectObj.Group(group_name)
ret = SapModel.GroupDefProp(group_name, property_name)

# Confirmar la selección
selected_items = []
ret, number_items, object_types, object_names = SapModel.SelectObj.GetSelected()
for i in range(number_items):
    selected_items.append((object_types[i], object_names[i]))

# Imprimir elementos seleccionados
print(f"Elementos seleccionados con la propiedad '{property_name}':")
for obj_type, obj_name in selected_items:
    print(f"Tipo: {obj_type}, Nombre: {obj_name}")

AttributeError: GroupDefProp

In [ ]:
#assign point object restraint at base

PointName1 = ' '

PointName2 = ' '

Restraint = [True, True, True, True, False, False]

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)

ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)

 

#assign point object restraint at top

Restraint = [True, True, False, False, False, False]

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)

ret = SapModel.PointObj.SetRestraint(PointName2, Restraint)

 

#refresh view, update (initialize) zoom

ret = SapModel.View.RefreshView(0, False)

In [8]:
#Crear Load Patterns 
LTYPE_DEAD = 1

LTYPE_SUPERDEAD = 2

LTYPE_LIVE = 3

LTYPE_REDUCELIVE = 4

LTYPE_QUAKE = 5

LTYPE_WIND= 6

LTYPE_SNOW = 7

LTYPE_OTHER = 8

LTYPE_MOVE = 9

LTYPE_TEMPERATURE = 10

LTYPE_ROOFLIVE = 11

LTYPE_NOTIONAL = 12

LTYPE_PATTERNLIVE = 13

LTYPE_WAVE= 14

LTYPE_BRAKING = 15

LTYPE_CENTRIFUGAL = 16

LTYPE_FRICTION = 17

LTYPE_ICE = 18

LTYPE_WINDONLIVELOAD = 19

LTYPE_HORIZONTALEARTHPRESSURE = 20

LTYPE_VERTICALEARTHPRESSURE = 21

LTYPE_EARTHSURCHARGE = 22

LTYPE_DOWNDRAG = 23

LTYPE_VEHICLECOLLISION = 24

LTYPE_VESSELCOLLISION = 25

LTYPE_TEMPERATUREGRADIENT = 26

LTYPE_SETTLEMENT = 27

LTYPE_SHRINKAGE = 28

LTYPE_CREEP = 29

LTYPE_WATERLOADPRESSURE = 30

LTYPE_LIVELOADSURCHARGE = 31

LTYPE_LOCKEDINFORCES = 32

LTYPE_PEDESTRIANLL = 33

LTYPE_PRESTRESS = 34

LTYPE_HYPERSTATIC = 35

LTYPE_BOUYANCY = 36

LTYPE_STREAMFLOW = 37

LTYPE_IMPACT = 38

LTYPE_CONSTRUCTION = 39

ret = SapModel.LoadPatterns.Add('1', LTYPE_OTHER,1, True)

ret = SapModel.LoadPatterns.Add('2', LTYPE_OTHER,0, True)

ret = SapModel.LoadPatterns.Add('3', LTYPE_LIVE, 0, True)

ret = SapModel.LoadPatterns.Add('4', 2, 0, True)

ret = SapModel.LoadPatterns.Add('5', 3, 0, True)

ret = SapModel.LoadPatterns.Add('6', LTYPE_OTHER,0, True)

ret = SapModel.LoadPatterns.Add('7', LTYPE_OTHER,0, True)


In [13]:
#Crear  Combinaciones

ret = SapModel.RespCombo.Add("ComboRenzo", 0)

In [16]:
ret = SapModel.RespCombo.SetCaseList("ComboRenzo", 0, "2", 1.6)

In [18]:
ret = SapModel.RespCombo.SetCaseList("ComboRenzo", 0, "31", 1.2)

In [12]:
#close Sap2000

ret = mySapObject.ApplicationExit(False)

SapModel = None

mySapObject = None
